In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely import wkt
import numpy as np
import swifter

from generator import RoadNetwork

In [2]:
df = pd.read_csv("../datasets/trajectories/Porto/mapped.csv")
df['POLYLINE'] = df['POLYLINE'].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry="POLYLINE")

Pandas Apply:   0%|          | 0/1641024 [00:00<?, ?it/s]

In [7]:
gdf.head()
gdf["POLYLINE"]

0          LINESTRING (-8.61864 41.14141, -8.61850 41.141...
1          LINESTRING (-8.63985 41.15983, -8.64035 41.159...
2          LINESTRING (-8.61296 41.14036, -8.61338 41.140...
3          LINESTRING (-8.57468 41.15195, -8.57470 41.151...
4          LINESTRING (-8.64599 41.18049, -8.64595 41.180...
                                 ...                        
1641019    LINESTRING (-8.60697 41.16228, -8.60697 41.162...
1641020    LINESTRING (-8.61247 41.14602, -8.61249 41.145...
1641021    LINESTRING (-8.61014 41.14084, -8.61017 41.140...
1641022    LINESTRING (-8.63071 41.15489, -8.63073 41.154...
1641023    LINESTRING (-8.61554 41.14063, -8.61542 41.140...
Name: POLYLINE, Length: 1641024, dtype: geometry

In [3]:
gdf["coords"] = gdf["POLYLINE"].swifter.apply(lambda x: list(x.coords))

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

In [9]:
gdf.head()

,Unnamed: 0,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,coords
0,0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"LINESTRING (-8.61864 41.14141, -8.61850 41.141...","[(-8.618643, 41.141412), (-8.618499, 41.141376..."
1,1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"LINESTRING (-8.63985 41.15983, -8.64035 41.159...","[(-8.639847, 41.159826), (-8.640351, 41.159871..."
2,2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"LINESTRING (-8.61296 41.14036, -8.61338 41.140...","[(-8.612964, 41.140359), (-8.613378, 41.14035)..."
3,3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"LINESTRING (-8.57468 41.15195, -8.57470 41.151...","[(-8.574678, 41.151951), (-8.574705, 41.151942..."
4,4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"LINESTRING (-8.64599 41.18049, -8.64595 41.180...","[(-8.645994, 41.18049), (-8.645949, 41.180517)..."


In [4]:
# Folium Heat Map
from folium import plugins
from folium.plugins import HeatMap

map_porto = folium.Map(location=[41.18049, -8.64599],
                    zoom_start = 10)

coords = gdf.iloc[:, -1].values
coords = np.concatenate(coords)
heat_data = [(c[1], c[0]) for c in coords]

# Plot it on the map
HeatMap(heat_data, radius=10).add_to(map_porto)

# Display the map
# map_porto
map_porto.save("heatmap_gps_points_porto.html")

Error: Canceled future for execute_request message before replies were done